# ELEC 598 : Deep Learning Applications 
# Project 1 - Predicting Housing Prices 
### By: Matthew Stein and Ethan Tsao 
### February 10, 2020 

In [1]:
from keras import models
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


## Data Preprocessing 

Data is imported via pandas, and processed via numpy. Working with Keras, all data is processed as arrays in numpy. To simplify model learning, feature-wise normalization is performed. Note that feature-wise normalization is calc'ed using the mean and std of the training set. 

The test set was chosen to be 15% of the overall data set. 

In [50]:
data = pd.read_excel('data/resd_data.xlsx')
length = data.shape[0]

# Remove all rows with null points in dataset 
updated_data = updated_data[np.isfinite(updated_data)]

# Remove all rows with "0" for the Postal Code, "0" for Year Built, "0" for Lot Square Footage, and "0" for List Price
updated_data = updated_data[updated_data.field_PostalCode != 0 ]
updated_data = updated_data[updated_data.YearBuilt != 0]
updated_data = updated_data[updated_data.LotSquareFootage != 0]
updated_data = updated_data[updated_data.ListPrice != 0]

# Update Index column 

print(updated_data) 

      SquareFootageStructure  LotSquareFootage  YearBuilt  Bedrooms  \
0                     1549.0            5825.0     1974.0       3.0   
1                     1196.0            7900.0     1981.0       3.0   
3                     3884.0           16013.0     1978.0       3.0   
6                     1164.0            6611.0     1961.0       3.0   
12                    2766.0            7331.0     1986.0       4.0   
...                      ...               ...        ...       ...   
9989                  2300.0            6547.0     1924.0       4.0   
9990                  1545.0            2178.0     1967.0       2.0   
9991                  1404.0            7841.0     1964.0       2.0   
9993                   900.0            6000.0     1924.0       2.0   
9999                  2865.0            7841.0     2005.0       4.0   

      BathsTotal  field_StoriesTotal  field_PostalCode  ListPrice  
0            2.0                 1.0           92624.0     3300.0  
1          

In [51]:
train_data = updated_data.loc[:5000,:'field_PostalCode'].values.tolist()
train_data = np.asarray(train_data) 
train_targets = updated_data.loc[:5000,'ListPrice'].values.tolist()
train_targets = np.asarray(train_targets) 

test_data = updated_data.loc[5001:,:'field_PostalCode'].values.tolist()
test_data = np.asarray(test_data) 
test_targets = updated_data.loc[5001:,'ListPrice'].values.tolist()
test_targets = np.asarray(test_targets)

In [52]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

## Model Selection 

As is standard for neural networks - and regression models, in particular - a Sequential model was chosen. The model has 3 hidden layers, with 64 

In [53]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

## Training and Validation 

In [ ]:
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_mae_histories = []


for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    print(train_data[:i * num_val_samples].shape)
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=32, verbose=1)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    test_mse, test_mae = model.evaluate(test_data, test_targets, verbose =1 ) 
    #mae_history = history.history['val_mean_absolute_error']
    #all_mae_histories.append(mae_history)

processing fold # 0
(0, 7)
Train on 2384 samples, validate on 794 samples
Epoch 1/100
2384/2384 [==============================] - 1s 383us/step - loss: 3396892369060.9395 - mae: 881247.1250 - val_loss: 2162124797482.5591 - val_mae: 851137.5625
Epoch 2/100
2384/2384 [==============================] - 0s 88us/step - loss: 3396345325554.2549 - mae: 881060.0000 - val_loss: 2161294233342.0654 - val_mae: 850781.3125
Epoch 3/100
2384/2384 [==============================] - 0s 88us/step - loss: 3394859532947.7583 - mae: 880509.6875 - val_loss: 2159332167876.0303 - val_mae: 849964.5000
Epoch 4/100
2384/2384 [==============================] - 0s 91us/step - loss: 3391122821972.1880 - mae: 879266.9375 - val_loss: 2155169980312.8262 - val_mae: 848365.5625
Epoch 5/100
2384/2384 [==============================] - 0s 93us/step - loss: 3384597323281.1812 - mae: 877301.8750 - val_loss: 2148127463264.0806 - val_mae: 846160.8750
Epoch 6/100
2384/2384 [==============================] - 0s 93us/step - los

2384/2384 [==============================] - 0s 102us/step - loss: 1812819935451.9194 - mae: 573777.2500 - val_loss: 1020672807603.2645 - val_mae: 552889.5000
Epoch 49/100
2384/2384 [==============================] - 0s 89us/step - loss: 1808994140819.7583 - mae: 573861.5625 - val_loss: 1019638528190.8716 - val_mae: 551696.5000
Epoch 50/100
2384/2384 [==============================] - 0s 88us/step - loss: 1806126272704.4297 - mae: 572643.4375 - val_loss: 1019085322967.3754 - val_mae: 551421.6875
Epoch 51/100
2384/2384 [==============================] - 0s 86us/step - loss: 1802640290919.0874 - mae: 572159.4375 - val_loss: 1018713349834.4786 - val_mae: 551055.5000
Epoch 52/100
2384/2384 [==============================] - 0s 88us/step - loss: 1801214756355.4363 - mae: 571685.6250 - val_loss: 1018217740478.8716 - val_mae: 550617.3125
Epoch 53/100
2384/2384 [==============================] - 0s 86us/step - loss: 1798168127144.3757 - mae: 570493.7500 - val_loss: 1017334055902.4685 - val_mae

Epoch 96/100
2384/2384 [==============================] - 0s 93us/step - loss: 1748979080831.1411 - mae: 551366.5000 - val_loss: 1025574490927.0730 - val_mae: 541939.4375
Epoch 97/100
2384/2384 [==============================] - 0s 95us/step - loss: 1747641682620.9934 - mae: 551613.8750 - val_loss: 1023841216094.1461 - val_mae: 540899.5625
Epoch 98/100
2384/2384 [==============================] - 0s 94us/step - loss: 1746455376889.1274 - mae: 551646.9375 - val_loss: 1027011518402.0957 - val_mae: 542615.1250
Epoch 99/100
2384/2384 [==============================] - 0s 94us/step - loss: 1746489420312.0537 - mae: 552027.9375 - val_loss: 1027257356855.4559 - val_mae: 542266.3125
Epoch 100/100
3192/3192 [==============================] - 0s 34us/step
processing fold # 1
(794, 7)
Train on 2384 samples, validate on 794 samples
Epoch 1/100
2384/2384 [==============================] - 1s 356us/step - loss: 2984684015148.6714 - mae: 875077.9375 - val_loss: 3399815291065.7129 - val_mae: 869676.18

Epoch 43/100
2384/2384 [==============================] - 0s 93us/step - loss: 1597173640363.8120 - mae: 569192.1875 - val_loss: 1650045204423.2544 - val_mae: 552380.1875
Epoch 44/100
2384/2384 [==============================] - 0s 89us/step - loss: 1595725501048.2686 - mae: 566654.9375 - val_loss: 1645501088734.4685 - val_mae: 551911.5625
Epoch 45/100
2384/2384 [==============================] - 0s 92us/step - loss: 1592707833691.0603 - mae: 566323.9375 - val_loss: 1644748858272.5642 - val_mae: 550860.0000
Epoch 46/100
2384/2384 [==============================] - 0s 92us/step - loss: 1590541970933.6912 - mae: 566144.9375 - val_loss: 1640626817864.8665 - val_mae: 550681.5000
Epoch 47/100
2384/2384 [==============================] - 0s 93us/step - loss: 1587379289672.1611 - mae: 565197.1875 - val_loss: 1640261744046.7507 - val_mae: 549422.2500
Epoch 48/100
2384/2384 [==============================] - 0s 94us/step - loss: 1584749644367.0334 - mae: 563773.0625 - val_loss: 1640212816222.79

## Results Discussion 